In [1]:
import sqlite3
import pandas as pd

import os, sys
rootpath = os.path.join(os.getcwd(), '.')
sys.path.append(rootpath)
from src.utils import *
# reimport modules if they change
%load_ext autoreload
%autoreload 2

In [2]:
# connect to the database (create file if not exists)
con = sqlite3.connect('./simulations.db')
# create a cursor object
cur = con.cursor()

## Simulations

In [3]:
# drop table if exists
#cur.execute('DROP TABLE IF EXISTS simulations')

In [4]:
# create table that does not allow duplicates
cur.execute("""
CREATE TABLE IF NOT EXISTS simulations (
            simulation_id integer PRIMARY KEY,
            N int NOT NULL,
            K int NOT NULL,
            lambda float NOT NULL,
            mu float NOT NULL,
            h float NOT NULL,
            window float NOT NULL,
            seed int NOT NULL,
            raw_file TEXT NOT NULL,
            dataset TEXT NOT NULL
            )
""")
cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS parameters ON simulations (N,K,lambda,mu,h,window,seed);")
# can be dropped with cur.execute("DROP INDEX parameters;") if needs to be redesigned
con.commit()

In [5]:
# show the simulations
# cur.execute("SELECT * FROM simulations")
# print(cur.fetchall())
simulations = pd.read_sql_query("SELECT * FROM simulations", con)
simulations

,simulation_id,N,K,lambda,mu,h,window,seed,raw_file,dataset
0,1,10000,100,0.0000,0.2,1.000000,0.000001,1001,/scratch02.local/johannes/projects/sahel_finit...,samples/1e-06
1,2,10000,100,0.0000,0.2,1.000000,1.000000,1001,/scratch02.local/johannes/projects/sahel_finit...,samples/1.0
2,3,10000,100,0.0000,0.2,1.000000,10.000000,1001,/scratch02.local/johannes/projects/sahel_finit...,samples/10.0
3,4,10000,100,0.0000,0.2,1.000000,100.000000,1001,/scratch02.local/johannes/projects/sahel_finit...,samples/100.0
4,5,10000,100,0.0000,0.2,1.000000,1000.000000,1001,/scratch02.local/johannes/projects/sahel_finit...,samples/1000.0
...,...,...,...,...,...,...,...,...,...,...
30031,30032,10000,100,0.9999,0.2,0.001778,1.000000,1008,/scratch02.local/johannes/projects/sahel_finit...,samples/1.0
30032,30033,10000,100,0.9999,0.2,0.001778,10.000000,1008,/scratch02.local/johannes/projects/sahel_finit...,samples/10.0
30033,30034,10000,100,0.9999,0.2,0.001778,100.000000,1008,/scratch02.local/johannes/projects/sahel_finit...,samples/100.0
30034,30035,10000,100,0.9999,0.2,0.001778,1000.000000,1008,/scratch02.local/johannes/projects/sahel_finit...,samples/1000.0


In [6]:
import numpy as np
unique_seeds = np.sort(simulations.seed.unique())
print("simulation seeds: ", unique_seeds)

# check that all simulations have entered the database
unique_lambda = np.sort(simulations["lambda"].unique())
unique_h = np.sort(simulations["h"].unique())
for seed in unique_seeds:
    for l in unique_lambda:
        for h in unique_h:
            windows = simulations[(simulations["seed"] == seed) & (simulations["lambda"] == l) & (simulations["h"] == h)]["window"].unique()
            # len(windows) should be 5 or 6 (seed1001)
            if len(windows) != 5:
                if len(windows) == 6 and seed == 1001:
                    continue
                else:
                    print(f"error for seed = {seed}, lambda = {l}, h = {h} : windows = {windows}")

simulation seeds:  [1000 1001 1002 1003 1004 1005 1006 1007 1008 1009]
error for seed = 1007, lambda = 0.0, h = 3.1622776601683795e-05 : windows = [   10.   100.  1000. 10000.]
error for seed = 1007, lambda = 0.0, h = 0.00031622776601683794 : windows = [   10.   100.  1000. 10000.]
error for seed = 1007, lambda = 0.0, h = 0.005623413251903491 : windows = [   10.   100.  1000. 10000.]
error for seed = 1007, lambda = 0.0, h = 1.0 : windows = [   10.   100.  1000. 10000.]
error for seed = 1007, lambda = 0.0, h = 5.623413251903491 : windows = [   10.   100.  1000. 10000.]
error for seed = 1007, lambda = 0.4376586748096509, h = 0.01778279410038923 : windows = [   10.   100.  1000. 10000.]
error for seed = 1007, lambda = 0.4376586748096509, h = 0.5623413251903491 : windows = [   10.   100.  1000. 10000.]
error for seed = 1007, lambda = 0.4376586748096509, h = 31.622776601683793 : windows = [   10.   100.  1000. 10000.]
error for seed = 1007, lambda = 0.4376586748096509, h = 100.0 : windows =

## Beta Approximation

In [39]:
# create table that does not allow duplicates
cur.execute("""
CREATE TABLE IF NOT EXISTS beta_approximations (
            simulation_id INTEGER NOT NULL UNIQUE,
            a float NOT NULL,
            b float NOT NULL,
            loc float NOT NULL,
            scale float NOT NULL,
            FOREIGN KEY (simulation_id) REFERENCES simulations (simulation_id)
            )
""")
con.commit()

In [57]:
# show state of approximation simulations
beta_approximations = pd.read_sql_query("SELECT * FROM beta_approximations", con)
beta_approximations

,simulation_id,a,b,loc,scale
0,1,3.000801e+03,2.072159e+04,-0.0001,1.0002
1,2,6.499563e+03,4.488178e+04,-0.0001,1.0002
2,3,5.995206e+04,4.139902e+05,-0.0001,1.0002
3,4,5.947034e+05,4.106638e+06,-0.0001,1.0002
4,5,5.693645e+06,3.931665e+07,-0.0001,1.0002
...,...,...,...,...,...
21347,21172,5.823130e+03,2.914583e+00,-0.0001,1.0002
21348,21173,1.590919e+04,7.962801e+00,-0.0001,1.0002
21349,21174,1.019550e+05,5.102996e+01,-0.0001,1.0002
21350,21175,7.111145e+05,3.559241e+02,-0.0001,1.0002


In [53]:
# check if approximations are available for all simulation_ids
# for simulation_id in simulations.simulation_id:
#     if np.sum(beta_approximations.simulation_id==simulation_id) != 1:
#         print(f"error: not a unique approximation for simulation_id = {simulation_id} with parameters:\n{simulations[simulations.simulation_id == simulation_id].iloc[0]}")

,simulation_id,a,b,loc,scale
20642,21352,4298.708376,17.629525,-0.0001,1.0002


## Beta Interpolation

In [41]:
# create table that does not allow duplicates
cur.execute("""
CREATE TABLE IF NOT EXISTS beta_interpolations (
            N INTEGER NOT NULL,
            K INTEGER NOT NULL,
            mu FLOAT NOT NULL,
            seed INTEGER NOT NULL,
            filename TEXT NOT NULL
            )
""")
cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS interpolation ON beta_interpolations (N,K,mu,seed);")
con.commit()

In [42]:
# show state of approximation simulations
beta_interpolations = pd.read_sql_query("SELECT * FROM beta_interpolations", con)
beta_interpolations

,N,K,mu,seed,filename
0,10000,100,0.2,1001,./dat/beta_interpolation_N=10000_K=100_mu=0.2_...
1,10000,100,0.2,1000,./dat/beta_interpolation_N=10000_K=100_mu=0.2_...
2,10000,100,0.2,1002,./dat/beta_interpolation_N=10000_K=100_mu=0.2_...
3,10000,100,0.2,1003,./dat/beta_interpolation_N=10000_K=100_mu=0.2_...
4,10000,100,0.2,1004,./dat/beta_interpolation_N=10000_K=100_mu=0.2_...
5,10000,100,0.2,1005,./dat/beta_interpolation_N=10000_K=100_mu=0.2_...
6,10000,100,0.2,1006,./dat/beta_interpolation_N=10000_K=100_mu=0.2_...


## Results

In [43]:
# create table that does not allow duplicates
cur.execute("""
CREATE TABLE IF NOT EXISTS results (
            N INTEGER NOT NULL,
            K INTEGER NOT NULL,
            mu FLOAT NOT NULL,
            seed INTEGER NOT NULL,
            window float NOT NULL,
            sigma float NOT NULL,
            epsilon float NOT NULL,
            filename TEXT NOT NULL
            )
""")
# do not allow duplicates
cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS result ON results (N,K,mu,seed,window,sigma,epsilon);")
con.commit()

In [44]:
results = pd.read_sql_query("SELECT * FROM results", con)
results

,N,K,mu,seed,window,sigma,epsilon,filename
0,10000,100,0.2,1003,1.000000,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...
1,10000,100,0.2,1003,10.000000,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...
2,10000,100,0.2,1003,100.000000,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...
3,10000,100,0.2,1003,1000.000000,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...
4,10000,100,0.2,1003,10000.000000,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...
...,...,...,...,...,...,...,...,...
114,10000,100,0.2,1006,1584.893192,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...
115,10000,100,0.2,1006,2511.886432,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...
116,10000,100,0.2,1006,3981.071706,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...
117,10000,100,0.2,1006,6309.573445,0.01,0.1,/data.nst/johannes/projects/sahel_finite-obser...


In [7]:
con.commit()
con.close()

## Clean up 

In [9]:
from tqdm import tqdm
import h5py
import numpy as np

know_what_you_do = True
if know_what_you_do:
    missing=0
    database = "./simulations.db"
    con = sqlite3.connect(database)
    cur = con.cursor()
    simulations = pd.read_sql_query("SELECT * FROM simulations", con)
    # check that all simulations have correct windows and datasets
    windows=[1e0,1e1,1e2,1e3,1e4]

    unique_seeds = [1007,1008,1009]
    print("simulation seeds: ", unique_seeds)
    for seed in tqdm(unique_seeds):
        dbseed = simulations[(simulations.seed == seed)]
        # unique lambda values
        unique_lambdas = np.sort(dbseed["lambda"].unique())
        for lam in unique_lambdas:
            dblam = dbseed[(dbseed["lambda"] == lam)]
            # unique h values
            unique_hs = np.sort(dblam["h"].unique())
            for h in unique_hs:
                sim = dblam[(dblam["h"] == h)]
                file_sim = sim["raw_file"].values[0]
                with h5py.File(file_sim, 'r') as f:
                    for window in windows:
                        try:
                            dset_db = sim[sim["window"]==window]["dataset"].values[0]
                        except IndexError: 
                            missing+=1
                            print(f"window {window} not found for seed {seed}, lambda {lam}, h {h}")
                            # check if dataset exists in raw file
                            try:
                                dset_std = f'samples/{window:.1f}'                    
                                data = f[dset_std]
                                print(f"{dset_std} found: {data}")
                                print(f"add to database: {sim.N.values[0]},{sim.K.values[0]},{sim['lambda'].values[0]},{sim.mu.values[0]},{sim.h.values[0]},{window},{sim.seed.values[0]},'{file_sim}','{dset_std}")
                                cur.execute(f"INSERT INTO simulations (N,K,lambda,mu,h,window,seed,raw_file,dataset) VALUES ({sim.N.values[0]},{sim.K.values[0]},{sim['lambda'].values[0]},{sim.mu.values[0]},{sim.h.values[0]},{window},{sim.seed.values[0]},'{file_sim}','{dset_std}')")
                            except KeyError:
                                print(f"window {window} not found in {file_sim}")
    if not missing:
        print("all datasets are set correctly")
    else:
        print(f"{missing} datasets were missing windows")
    con.commit()
    con.close()

simulation seeds:  [1007, 1008, 1009]


100%|██████████| 3/3 [00:03<00:00,  1.20s/it]

all datasets are set correctly


In [12]:
# test saving files with
if True:
    database = "./test.db"
    from src.simulation import *

    params={
        'N': int(1e2),
        'K': 10,
        'lambda': 0.0,
        'mu': 0.2,
        'h': 1,
        'seed': 1234,
    }
    result = simulation(params)
    save_simulation(result, path='./logs/', database=database, verbose=True)

[6.32120559e-01 9.51625820e-02 9.95016625e-03 9.99500167e-04
 9.99950002e-05]
# COMMENT: burn-in steps self-consistently set to 1.00e+04 = max(30 * tau, window_max) with tau = -dt / ln(lambda) = 0.00e+00 and window_max=1.00e+04
# COMMENT: equilibration steps self-consistently set to 3.00e+04 = 3 * window_max = 1.00e+04
# COMMENT: recording steps self-consistently set to 1.00e+06 = max(1000 * tau, 100*window_max) with tau = -dt / ln(lambda) = 0.00e+00 and window_max=1.00e+04
initialize with random spiking condition


burn-in dynamics:   0%|          | 0/10000 [00:00<?, ?it/s]

recording: 100%|██████████| 1000000/1000000 [00:39<00:00, 25162.07it/s]


hdf5 saving samples/1.0
hdf5 saving samples/10.0
hdf5 saving samples/100.0
hdf5 saving samples/1000.0
hdf5 saving samples/10000.0
hdf5 saving steps/burn
hdf5 saving steps/equil
hdf5 saving steps/record
sqlite3 saving 1.0
sqlite3 saving 10.0
sqlite3 saving 100.0
sqlite3 saving 1000.0
sqlite3 saving 10000.0
